In [6]:
from kubernetes import client, config, stream
import re
import concurrent.futures
import time

# Load Kubernetes configuration
config.load_kube_config()

def measure_bandwidth_from_source_to_target(v1, namespace, source_pod, target_pod, test_duration=5):
    """Measure bandwidth from source pod to target pod."""
    source_name = source_pod.metadata.name
    target_ip = target_pod.status.pod_ip
    target_name = target_pod.metadata.name
    source_node = source_pod.spec.node_name
    target_node = target_pod.spec.node_name

    if source_name == target_name:
        return None  # Skip self-tests

    command = ['iperf3', '-c', target_ip, '-t', str(test_duration)]
    try:
        output = stream.stream(v1.connect_get_namespaced_pod_exec,
                               source_name,
                               namespace,
                               command=command,
                               stderr=True,
                               stdin=False,
                               stdout=True,
                               tty=False)
        # Parse the output for bandwidth
        match = re.search(r'(\d+\.?\d*\s[MKG]bits/sec)', output)
        bandwidth = match.group(1) if match else "Parsing Error"

        # Print the result immediately
        print(f"{source_node} -> {target_node}: {bandwidth}")
        return source_node, target_node, bandwidth

    except Exception as e:
        print(f"Error measuring bandwidth from {source_name} to {target_name}: {e}")
        return source_node, target_node, "Error"

def measure_bandwidth(namespace='measure-nodes-bd', max_concurrent_tasks=3, test_duration=5):
    """Measure bandwidth between all pods in the namespace."""
    v1 = client.CoreV1Api()
    pods = v1.list_namespaced_pod(namespace, label_selector="app=bandwidth-measurement").items

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_concurrent_tasks) as executor:
        futures = [
            executor.submit(measure_bandwidth_from_source_to_target, v1, namespace, src, tgt, test_duration)
            for src in pods for tgt in pods if src.metadata.name != tgt.metadata.name
        ]

        # Collect and print results as they complete
        results = {}
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                src_node, tgt_node, bandwidth = result
                if src_node not in results:
                    results[src_node] = {}
                results[src_node][tgt_node] = bandwidth

    return results

if __name__ == '__main__':
    namespace = 'measure-nodes-bd'
    print("Measuring bandwidth...")
    time.sleep(10)  # Allow some time for the DaemonSet to deploy
    results = measure_bandwidth(namespace=namespace, max_concurrent_tasks= 4, test_duration=10)
    print("\nFinal Cross-Node Bandwidth Results:")
    for src_node, targets in results.items():
        for tgt_node, bandwidth in targets.items():
            print(f"{src_node} -> {tgt_node}: {bandwidth}")


Measuring bandwidth...
k8s-worker-3 -> k8s-worker-5: 392 Mbits/sec
k8s-worker-3 -> k8s-worker-1: 647 Mbits/sec
k8s-worker-3 -> k8s-worker-8: 362 Mbits/sec
k8s-worker-3 -> k8s-worker-9: 295 Mbits/sec
k8s-worker-3 -> k8s-worker-2: 513 Mbits/sec
k8s-worker-3 -> k8s-worker-6: 527 Mbits/sec
k8s-worker-3 -> k8s-worker-7: 461 Mbits/sec
k8s-worker-3 -> k8s-worker-4: 269 Mbits/sec
k8s-worker-1 -> k8s-worker-5: 642 Mbits/sec
k8s-worker-1 -> k8s-worker-3: 408 Mbits/sec
k8s-worker-1 -> k8s-worker-8: 306 Mbits/sec
k8s-worker-1 -> k8s-worker-9: 733 Mbits/sec
k8s-worker-1 -> k8s-worker-6: 487 Mbits/sec
k8s-worker-1 -> k8s-worker-7: 742 Mbits/sec
k8s-worker-1 -> k8s-worker-2: 431 Mbits/sec
k8s-worker-1 -> k8s-worker-4: 232 Mbits/sec
k8s-worker-8 -> k8s-worker-3: 488 Mbits/sec
k8s-worker-8 -> k8s-worker-1: 740 Mbits/sec
k8s-worker-8 -> k8s-worker-5: 229 Mbits/sec
k8s-worker-8 -> k8s-worker-9: 452 Mbits/sec
k8s-worker-8 -> k8s-worker-6: 495 Mbits/sec
k8s-worker-8 -> k8s-worker-2: 259 Mbits/sec
k8s-worke

In [8]:
import pandas as pd

# Convert the nested dictionary into a pandas DataFrame
df = pd.DataFrame(results)

# Transpose the DataFrame to align the source and destination workers as per convention
df = df.T

# Fill diagonal with 0's for self-latency (optional, if desired for clarity)
for worker in df.columns:
    df.at[worker, worker] = 0

# Save the DataFrame to a CSV file, and add the date, hour and minute to the file name
import datetime
now = datetime.datetime.now()
filename = f"bandwidth_results_{now.strftime('%Y-%m-%d_%H-%M')}.csv"
df.to_csv("/home/ubuntu/iDynamics/iBandwidth/measurer/data/"+filename)
df

,k8s-worker-5,k8s-worker-1,k8s-worker-8,k8s-worker-9,k8s-worker-2,k8s-worker-6,k8s-worker-7,k8s-worker-4,k8s-worker-3
k8s-worker-3,392 Mbits/sec,647 Mbits/sec,362 Mbits/sec,295 Mbits/sec,513 Mbits/sec,527 Mbits/sec,461 Mbits/sec,269 Mbits/sec,0
k8s-worker-1,642 Mbits/sec,0,306 Mbits/sec,733 Mbits/sec,431 Mbits/sec,487 Mbits/sec,742 Mbits/sec,232 Mbits/sec,408 Mbits/sec
k8s-worker-8,229 Mbits/sec,740 Mbits/sec,0,452 Mbits/sec,259 Mbits/sec,495 Mbits/sec,675 Mbits/sec,393 Mbits/sec,488 Mbits/sec
k8s-worker-5,0,629 Mbits/sec,739 Mbits/sec,266 Mbits/sec,365 Mbits/sec,613 Mbits/sec,480 Mbits/sec,663 Mbits/sec,582 Mbits/sec
k8s-worker-9,491 Mbits/sec,256 Mbits/sec,389 Mbits/sec,0,737 Mbits/sec,268 Mbits/sec,377 Mbits/sec,647 Mbits/sec,767 Mbits/sec
k8s-worker-6,383 Mbits/sec,337 Mbits/sec,634 Mbits/sec,500 Mbits/sec,644 Mbits/sec,0,545 Mbits/sec,475 Mbits/sec,786 Mbits/sec
k8s-worker-2,347 Mbits/sec,406 Mbits/sec,332 Mbits/sec,720 Mbits/sec,0,450 Mbits/sec,526 Mbits/sec,279 Mbits/sec,623 Mbits/sec
k8s-worker-7,578 Mbits/sec,264 Mbits/sec,456 Mbits/sec,716 Mbits/sec,427 Mbits/sec,657 Mbits/sec,0,571 Mbits/sec,501 Mbits/sec
k8s-worker-4,609 Mbits/sec,258 Mbits/sec,694 Mbits/sec,238 Mbits/sec,569 Mbits/sec,438 Mbits/sec,545 Mbits/sec,0,247 Mbits/sec
